In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
import os,sys
from pathlib import Path
import json 

from io import StringIO
import pprint

import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

#Python files
from config import Configuration 

from utils import *
from models_helper import *
from datasets import EclipseSampledDataset, VoltaSampledDataset, ODDSDataset
from models import BaseAutoencoder

from sklearn import svm   
from sklearn.linear_model import LogisticRegression  

In [56]:
def runMultipleTrainTestsScikit(name,model_config,X_train,y_train,X_test,y_test,repeat,size,):
        from sklearn.ensemble import RandomForestClassifier                                        
                
        macro_fscore = 0
        weight_fscore = 0
        miss_rate = 0
        alarm_rate = 0
        score_dict = {}
        
        for i in range(repeat):
            
            if name == 'tuncer_rf-fs':

                clf = RandomForestClassifier(n_estimators=100)
                clf.fit(X_train, y_train)      
                
                
            test_pred_label = clf.predict(X_test)
            score_report = classification_report(y_true=y_test, y_pred =test_pred_label,output_dict=True)
            logging.info("RF Report")
            logging.info(classification_report(y_true=y_test, y_pred =test_pred_label))
            alarm_report = falseAnomRateCalc(y_test,test_pred_label)
            
            macro_fscore += score_report['macro avg']['f1-score']            
            weight_fscore += score_report['weighted avg']['f1-score']
            miss_rate += alarm_report['anom_miss_rate'] 
            alarm_rate += alarm_report['false_alarm_rate']
        
        
        score_dict['false_alarm_rate'] = alarm_rate / repeat
        score_dict['anom_miss_rate'] = miss_rate / repeat        
        score_dict['macro_fscore'] = macro_fscore / repeat
        score_dict['weight_fscore'] = weight_fscore / repeat
        
        
        name = (name + "_test_{}").format(size)
        json_dump = json.dumps(score_dict)
        f_json = open(model_config['results_dir'] / ("{}_score_dict.json".format(name)),"w")
        f_json.write(json_dump)
        f_json.close()                                        

In [57]:
def runMultipleTrainTestsProctor(name,params,model_config,normal_name,anom_name,all_name,X_train,y_train,X_test,y_test,repeat,size,freeze=False,multimodal=True,stacked=False):
                        
        macro_fscore = 0
        weight_fscore = 0
        miss_rate = 0
        alarm_rate = 0
        score_dict = {}
        
        for i in range(repeat):
                        
            if multimodal:
            
                #Only anom vs normal            
                train_binary_label = y_train.copy()
                train_binary_label[y_train != 0] = 1         
                
                binary_model = trainSupervisedAE(params,
                                                  model_config,normal_name,
                                                  X_train,train_binary_label,freeze=freeze,stacked=stacked)   

                #Only anomalies
                anom_indices = np.where(y_train != 0)
                train_anom_label = y_train[anom_indices]
                train_anom_data = X_train[anom_indices]
                
                multiclass_anom_model = trainSupervisedAE(params,
                                                  model_config,anom_name,
                                                  train_anom_data,train_anom_label,freeze=freeze,stacked=stacked)                  
                
                #Double-layer testing
                test_pred_label = filteredTestingProctor(binary_model,multiclass_anom_model,X_test)
            
            
            else:
                #All labels / all-in-one         
                multiclass_model = trainSupervisedAE(params,
                                                  model_config,all_name,
                                                  X_train,y_train,freeze=freeze)             
            
                test_pred_label = np.argmax(multiclass_model.predict(X_test),1)
                
            score_report = classification_report(y_true=y_test, y_pred =test_pred_label,output_dict=True)
            #print(score_report)
            alarm_report = falseAnomRateCalc(y_test,test_pred_label)

            macro_fscore += score_report['macro avg']['f1-score']            
            weight_fscore += score_report['weighted avg']['f1-score']
            miss_rate += alarm_report['anom_miss_rate'] 
            alarm_rate += alarm_report['false_alarm_rate']

            score_dict['false_alarm_rate'] = alarm_rate / repeat
            score_dict['anom_miss_rate'] = miss_rate / repeat        
            score_dict['macro_fscore'] = macro_fscore / repeat
            score_dict['weight_fscore'] = weight_fscore / repeat            
          
        name = (name + "_test_{}").format(size)
        json_dump = json.dumps(score_dict)
        f_json = open(model_config['results_dir'] / ("{}_score_dict.json".format(name)),"w")
        f_json.write(json_dump)
        f_json.close()         
        
        
def runMultipleTrainTestsProctorScikit(name,model_config,topology,X_train,y_train,X_test,y_test,repeat,size,multimodal=True,stacked=False):
      
                                      
        macro_fscore = 0
        weight_fscore = 0
        miss_rate = 0
        alarm_rate = 0
        score_dict = {}
        
        #It means loading a full autoencoder
        if not stacked:
            reconstructedModel = LoadModel(model_config,topology)
            reconstructedModel = reconstructedModel.encoder
        else:
            reconstructedModel = LoadModel(model_config,topology)
                
        #Test data is constant 
        hidden_test = reconstructedModel.predict(X_test)             
        
#         param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#                       'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#                       'class_weight': [None, 'balanced'],
#                       'kernel': ['rbf','linear','sigmoid','poly']}  
        
        param_grid = {'C': [0.1,0.5, 1, 5, 10, 100, 1000],  
                      'tol': [1e-4, 1e-5, 1e-6,1e-7], 
                      'class_weight': [None, 'balanced'],
                      'max_iter': [1000,3000,5000,10000],
                      'penalty': ['l1','l2']
                      
                      }  
                    
        for i in range(repeat):                                    
            
            if multimodal:
                
                #Only anom vs normal            
                train_binary_label = y_train.copy()
                train_binary_label[train_binary_label != 0] = 1  
                                
                hidden_train_binary = reconstructedModel.predict(X_train)
                                

#                 grid = GridSearchCV(svm.LinearSVC(), param_grid, refit = True, verbose = 3) 
#                 grid.fit(hidden_train_binary, train_binary_label)     
#                 logging.info("BEST binary")
#                 logging.info(grid.best_estimator_)
#                 clf_binary = grid.best_estimator_
                
                ####Previous
                clf_binary = svm.LinearSVC(max_iter=10000,C=100)  
#                 clf_binary = svm.LinearSVC(C=0.1, 
#                                            #class_weight='balanced', 
#                                            dual=True, fit_intercept=True,
#                   intercept_scaling=1, loss='squared_hinge', max_iter=1000,
#                   multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
#                   verbose=0)                
                clf_binary.fit(hidden_train_binary, train_binary_label)                
                
                #Only anomalies
                anom_indices = np.where(y_train != 0)
                train_anom_label = y_train[anom_indices]
                train_anom_data = X_train[anom_indices]  
                
                hidden_train_anom = reconstructedModel.predict(train_anom_data)
                
#                 grid = GridSearchCV(svm.LinearSVC(), param_grid, refit = True, verbose = 3) 
#                 grid.fit(hidden_train_anom, train_anom_label)                     
#                 logging.info("BEST multi")                
#                 logging.info(grid.best_estimator_)                
#                 clf_anom = grid.best_estimator_   
                
                ###Previous
#                 clf_anom = svm.LinearSVC(C=5, 
#                                          #class_weight='balanced', 
#                                          dual=True, fit_intercept=True,
#                   intercept_scaling=1, loss='squared_hinge', max_iter=1000,
#                   multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
#                   verbose=0)                
                clf_anom = svm.LinearSVC(max_iter=10000,C=100)    
                clf_anom.fit(hidden_train_anom, train_anom_label)                
                
                test_pred_label = filteredTestingProctorScikit(clf_binary,clf_anom,hidden_test)
            
            else:
                if name == 'aksar_svm':
                    hidden_train = reconstructedModel.predict(X_train)
                    clf = svm.LinearSVC(max_iter=10000,C=0.4)    
                    #clf = svm.LinearSVC()    
                    clf.fit(hidden_train, y_train)                            
                    test_pred_label = clf.predict(hidden_test)
                    
                elif name == 'aksar_lr':      
                    
                    hidden_train = reconstructedModel.predict(X_train)
                    clf = LogisticRegression(random_state=1234,
                                                    dual=True,
                                                    penalty='l2',
                                                    solver='liblinear')                      
                    clf.fit(hidden_train, y_train)                            
                    test_pred_label = clf.predict(hidden_test)
                    
                elif name == 'aksar_svm-scale':      
                    hidden_train = reconstructedModel.predict(X_train)
                    clf = svm.SVC(C=1,kernel='rbf',probability=True)
                    clf.fit(hidden_train, y_train)                            
                    test_pred_label = clf.predict(hidden_test)                                                                                             
            
            score_report = classification_report(y_true=y_test, y_pred =test_pred_label,output_dict=True)
            logging.info("#############")
            logging.info(classification_report(y_true=y_test, y_pred =test_pred_label))
            logging.info("#############")            
            alarm_report = falseAnomRateCalc(y_test,test_pred_label)
            
            macro_fscore += score_report['macro avg']['f1-score']            
            weight_fscore += score_report['weighted avg']['f1-score']
            miss_rate += alarm_report['anom_miss_rate'] 
            alarm_rate += alarm_report['false_alarm_rate']
        
        
        score_dict['false_alarm_rate'] = alarm_rate / repeat
        score_dict['anom_miss_rate'] = miss_rate / repeat        
        score_dict['macro_fscore'] = macro_fscore / repeat
        score_dict['weight_fscore'] = weight_fscore / repeat
        

        
        name = (name + "_test_{}").format(size)
        json_dump = json.dumps(score_dict)
        f_json = open(model_config['results_dir'] / ("{}_score_dict.json".format(name)),"w")
        f_json.write(json_dump)
        f_json.close()                   

In [58]:
# ###USE for stacked AE cases
# SCALER = 'MinMax'
# EXP_NAME = 'tuncer_baseline_10'

# MODEL_NAME = 'AE_1_stacked' #change model name when you are trying a new model
# SYSTEM = 'eclipse'

# TOPOLOGY_NAME = 'stacked_ae_2'
# STACKED = True #Set this when loading a stacked AE

# FEATURE_SELECTION = False
# REPEAT = 1

In [59]:
###USE for normal AE cases
SCALER = 'MinMax'

###Paper generation
EXP_NAME = 'tuncer_baseline_10'
MODEL_NAME = 'AE_5'

# EXP_NAME = 'anomaly_diagnosis_CR_Seed'
# MODEL_NAME = 'AE_CR'

SYSTEM = 'eclipse'

TOPOLOGY_NAME = 'dae'
STACKED = False #Set this when loading a stacked AE

FEATURE_SELECTION = False
REPEAT = 1

In [60]:
params_finetune = {}

#Compile related
params_finetune['model_name'] = 'sae_supervised-ft'

params_finetune['loss'] = 'categorical_crossentropy'

params_finetune['optimizer'] = 'adam'
params_finetune['learning_rate'] = 1e-3
params_finetune['dense_neuron'] = 250

#Training related
params_finetune['epochs'] = 150

In [63]:
if SYSTEM == 'eclipse':
    #labeled_data_percentages = list(reversed([0.1,0.09,0.08,0.07,0.06,0.05,0.04,0.03,0.022,0.02]))    
    labeled_data_percentages = list(reversed([0.15,0.20, 0.25,0.30]))    
    #labeled_data_percentages = list(reversed([0.022]))    
elif SYSTEM == 'volta':
    #labeled_data_percentages = [max_ratio,0.05,0.04,0.03,0.02,0.01,0.008,0.006,0.004,0.002]
    labeled_data_percentages = list(reversed([0.02,0.01,0.0065,0.0055,0.0050,0.0040,0.0035,0.0030,0.0025,0.0020,0.0016,0.0013,0.0010]))                    
    #labeled_data_percentages = [0.0050]

In [64]:
### TRAIN AND TEST 
#for CV_INDEX in [1,2,3,4]:
for CV_INDEX in [0,1,2,3,4]:


    model_config = readModelConfig(exp_name=EXP_NAME,cv_index=CV_INDEX,model_name=MODEL_NAME,system=SYSTEM)
    exp_config = readExperimentConfig(exp_name=EXP_NAME,system=SYSTEM)

    if SYSTEM == 'eclipse':

        import json
        with open(str(exp_config['experiment_dir']) + '/anom_dict.json') as f:
            ANOM_DICT = json.load(f)    

        eclipseDataset = EclipseSampledDataset(exp_config)
        train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(scaler=SCALER)
        assert list(train_data.index) == list(train_label.index) #check the order of the labels     
        assert list(test_data.index) == list(test_label.index) #check the order of the labels            

    elif SYSTEM == 'volta':

        import json
        with open(str(exp_config['experiment_dir']) + '/anom_dict.json') as f:
            ANOM_DICT = json.load(f)    

        voltaDataset = VoltaSampledDataset(exp_config)
        train_data, train_label, test_data, test_label = voltaDataset.load_dataset(scaler=SCALER)
        assert list(train_data.index) == list(train_label.index) #check the order of the labels     
        assert list(test_data.index) == list(test_label.index) #check the order of the labels    

    elif SYSTEM in ['thyroid','shuttle','satimage-2','satellite','cardio','arrhythmia']:

        dataset = ODDSDataset(root='/projectnb/peaclab-mon/aksar/nonhpc_datasets/ODDS',dataset_name=DATASET)
        train_data, train_label, test_data, test_label = dataset.load_dataset()     

    if FEATURE_SELECTION:
        selected_features = pd.read_csv(exp_config['experiment_dir'] / 'selected_features.csv')
        #Training data with feature selection
        train_data = train_data[list(selected_features['0'].values)]
        test_data = test_data[list(selected_features['0'].values)]        

    logging.info("Train data shape %s",train_data.shape)
    logging.info("Train label shape %s",train_label.shape)
    logging.info("Test data shape %s",test_data.shape)  
    logging.info("Test label shape %s",test_label.shape)

    logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
    logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())  
    
    selected_features = pd.read_csv(exp_config['experiment_dir'] / 'selected_features.csv')
    
    #Convert test data to array format
    assert list(test_label.index) == list(test_data.index.get_level_values('node_id')) 
    test_true_data = test_data.values 
    test_true_data_fs = test_data[list(selected_features['0'].values)] 
    test_true_label = test_label['anom'].values 

    #Training data with feature selection
    train_data_fs = train_data[list(selected_features['0'].values)]
    X_DIM = train_data.shape[1]

    FIRST_TIME = True    

    for PERCENTAGE in labeled_data_percentages:

        train_semisup_data, train_semisup_label = prepareFinetuningDataset(PERCENTAGE,
                                                                           train_data,
                                                                           train_label,
                                                                           SYSTEM)
        logging.info("NO Feature selected data shape %s",train_semisup_data.shape) 
        logging.info("PERCENTAGE %s",PERCENTAGE)
        logging.info("Train data label dist: \n%s",pd.DataFrame(train_semisup_label)[0].value_counts())
        logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        

        #DL MODELS
#         if FIRST_TIME:        
#             #Create two supervised AE's classifiers 
#             buildBinaryAE(params_finetune,model_config,TOPOLOGY_NAME,
#                           model_name='normal-sae',stacked=STACKED)   
#             #to_categorical generates encoding according to existing label, e.g., if label 4 exists, automatically 5 dimension
#             buildMulticlassAE(params_finetune,model_config,TOPOLOGY_NAME,num_class = 5,
#                               model_name='anomaly-sae',stacked=STACKED)      
#             buildMulticlassAE(params_finetune,model_config,TOPOLOGY_NAME,num_class = 5,
#                               model_name='all-sae',stacked=STACKED) 
#             FIRST_TIME = False
            
#         runMultipleTrainTestsProctor(
#                             'aksar_dnn-mm',
#                             params_finetune,
#                             model_config,
#                             'normal-sae',
#                             'anomaly-sae',
#                             'all-sae', #TODO: Not used in this scenario but need to provide 
#                             train_semisup_data,
#                             train_semisup_label,
#                             test_true_data,
#                             test_true_label,
#                             REPEAT,
#                             PERCENTAGE,
#                             freeze=False, #TODO: sth funky here when true
#                             multimodal=True,
#                             stacked=STACKED)
               
        
#         runMultipleTrainTestsProctor(
#                             'aksar_dnn',
#                             params_finetune,
#                             model_config,
#                             'normal-sae', #TODO: Not used in this scenario but need to provide 
#                             'anomaly-sae', #TODO: Not used in this scenario but need to provide 
#                             'all-sae', 
#                             train_semisup_data,
#                             train_semisup_label,
#                             test_true_data,
#                             test_true_label,
#                             REPEAT,
#                             PERCENTAGE,
#                             freeze=False, #TODO: sth funky here
#                             multimodal=False,
#                             stacked=STACKED)        
           
            
        runMultipleTrainTestsProctorScikit(      
                            'aksar_svm-mm',
                            model_config,
                            TOPOLOGY_NAME,
                            train_semisup_data,
                            train_semisup_label,
                            test_true_data,
                            test_true_label,
                            REPEAT,
                            PERCENTAGE,
                            multimodal=True, #TODO: multi modal SVM needs debug
                            stacked=STACKED)         
        
#         runMultipleTrainTestsProctorScikit(      
#                             'aksar_svm',
#                             model_config,
#                             TOPOLOGY_NAME,
#                             train_semisup_data,
#                             train_semisup_label,
#                             test_true_data,
#                             test_true_label,
#                             REPEAT,
#                             PERCENTAGE,
#                             multimodal=False,
#                             stacked=STACKED)    
        
        
#         runMultipleTrainTestsProctorScikit(      
#                             'aksar_lr',
#                             model_config,
#                             TOPOLOGY_NAME,
#                             train_semisup_data,
#                             train_semisup_label,
#                             test_true_data,
#                             test_true_label,
#                             REPEAT,
#                             PERCENTAGE,
#                             multimodal=False,
#                             stacked=STACKED)     
        
        
#         runMultipleTrainTestsProctorScikit(      
#                             'aksar_svm-scale',
#                             model_config,
#                             TOPOLOGY_NAME,
#                             train_semisup_data,
#                             train_semisup_label,
#                             test_true_data,
#                             test_true_label,
#                             REPEAT,
#                             PERCENTAGE,
#                             multimodal=False,
#                             stacked=STACKED)         
        
                        
        #FEATURE SELECTION
        logging.info("Analysis with feature selection")    
        train_semisup_data, train_semisup_label = prepareFinetuningDataset(PERCENTAGE,
                                                                           train_data,
                                                                           #train_data_fs,
                                                                           train_label,
                                                                           SYSTEM,rs=42)     
        logging.info("Feature selected data shape %s",train_semisup_data.shape)

        #RF-Tuncer WITH Feature Selection                 
        runMultipleTrainTestsScikit(      
                            'tuncer_rf-fs',
                            model_config,
                            train_semisup_data,
                            train_semisup_label,
                            test_true_data,            
                            #test_true_data_fs,
                            test_true_label,
                            REPEAT,
                            PERCENTAGE)             

2021-03-23 10:26:23,675 INFO    BaseDataset Class Initialization
2021-03-23 10:26:23,675 INFO    HPCDataset Class Initialization
2021-03-23 10:26:23,676 INFO    EclipseSampledDataset Class Initialization
2021-03-23 10:26:24,115 INFO    Train data shape (674, 1716)
2021-03-23 10:26:24,116 INFO    Train label shape (674, 2)
2021-03-23 10:26:24,116 INFO    Test data shape (3139, 1716)
2021-03-23 10:26:24,116 INFO    Test label shape (3139, 2)
2021-03-23 10:26:24,174 INFO    Train data shape (674, 1716)
2021-03-23 10:26:24,174 INFO    Train label shape (674, 2)
2021-03-23 10:26:24,174 INFO    Test data shape (3139, 1716)
2021-03-23 10:26:24,175 INFO    Test label shape (3139, 2)
2021-03-23 10:26:24,176 INFO    Train data label dist: 
0    607
3     17
2     17
1     17
4     16
Name: anom, dtype: int64
2021-03-23 10:26:24,178 INFO    Test data label dist: 
0    913
1    559
2    559
3    555
4    553
Name: anom, dtype: int64
2021-03-23 10:26:24,197 INFO    
Semi-supervised labeled data cla

40
16


2021-03-23 10:26:24,581 WARNING AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x2b041e306730> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-03-23 10:26:25,837 INFO    Double layer testing started
2021-03-23 10:26:25,908 INFO    #############
2021-03-23 10:26:25,913 INFO                  precision    recall  f1-score   support

           0       0.95      1.00      0.97       913
           1       0.66      0.58      0.61       559
           2       0.44      0.46      0.45       559
           3       0.44      0.49      0.47       555
           4       1.00      0.88      0.94       553

    accuracy                           0.72      3139
   macro avg       0.70      0.68      0.69      3139
weighted avg       0.73      0.72      0.72      3139

2021-03-23 10:26:25,914 INFO    #############
2021-03-23 10:26:25,915 INFO    Total misclassified normal runs: 1, Total normal runs 913 
2021-03-23 10:26:25,915 INFO    0.001095290251916758
2021-03-23 10:26:25,916 INFO    Total misclassified anom runs: 48, Total anom runs 2226 
2021-03-23 10:26:25,916 INFO    0.0215633423180593
2021-03-23 10:26:25,918 INFO    Analysis w

...

26
16
20
16


2021-03-23 10:27:02,879 WARNING AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x2b04848b20d0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-03-23 10:27:03,990 INFO    Double layer testing started
2021-03-23 10:27:04,074 INFO    #############
2021-03-23 10:27:04,080 INFO                  precision    recall  f1-score   support

           0       0.96      1.00      0.98       913
           1       0.62      0.58      0.60       559
           2       0.45      0.45      0.45       559
           3       0.44      0.51      0.47       555
           4       0.99      0.85      0.91       553

    accuracy                           0.71      3139
   macro avg       0.69      0.68      0.68      3139
weighted avg       0.72      0.71      0.72      3139

2021-03-23 10:27:04,080 INFO    #############
2021-03-23 10:27:04,081 INFO    Total misclassified normal runs: 2, Total normal runs 913 
2021-03-23 10:27:04,081 INFO    0.002190580503833516
2021-03-23 10:27:04,082 INFO    Total misclassified anom runs: 37, Total anom runs 2226 
2021-03-23 10:27:04,082 INFO    0.016621743036837375
2021-03-23 10:27:04,084 INFO    Analysis

20
16


In [ ]:
2021-03-23 08:27:49,506 INFO    BEST binary
2021-03-23 08:27:49,507 INFO    LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
2021-03-23 08:31:52,937 INFO    BEST multi
2021-03-23 08:31:52,938 INFO    LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
2021-03-23 01:06:35,553 INFO    BEST binary
2021-03-23 01:06:35,554 INFO    
        LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
2021-03-23 01:08:20,888 INFO    
        LinearSVC(C=10, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
          verbose=0)

In [267]:
train_semisup_data, train_semisup_label = prepareFinetuningDataset(0.07,
                                                                   train_data,
                                                                   train_label,
                                                                   SYSTEM)
logging.info("NO Feature selected data shape %s",train_semisup_data.shape) 
logging.info("PERCENTAGE %s",PERCENTAGE)
logging.info("Train data label dist: \n%s",pd.DataFrame(train_semisup_label)[0].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        


2021-03-23 01:10:18,588 INFO    
Semi-supervised labeled data class distributions
2    10
1    10
4     9
3     9
0     9
Name: anom, dtype: int64

2021-03-23 01:10:18,589 INFO    NO Feature selected data shape (47, 1716)
2021-03-23 01:10:18,590 INFO    PERCENTAGE 0.1
2021-03-23 01:10:18,591 INFO    Train data label dist: 
2    10
1    10
4     9
3     9
0     9
Name: 0, dtype: int64
2021-03-23 01:10:18,593 INFO    Test data label dist: 
0    913
1    559
2    559
3    556
4    553
Name: anom, dtype: int64


9
16


In [227]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 

from scipy.stats import reciprocal, uniform
from sklearn.svm import SVC


In [223]:
topology = 'dae'
reconstructedModel = LoadModel(model_config,topology)
reconstructedModel = reconstructedModel.encoder

#Test data is constant 
hidden_test = reconstructedModel.predict(test_true_data)     
hidden_train = reconstructedModel.predict(train_semisup_data)

2021-03-23 00:46:07,362 INFO    Loading model!
2021-03-23 00:46:08,891 WARNING AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x2b63c6091510> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [231]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'class_weight': ['balanced'],
              'kernel': ['rbf','linear','sigmoid','poly']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

grid.fit(hidden_train, train_semisup_label)     

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.200, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.200, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.222, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.222, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf ...............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, score=0.111, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=linear ............
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=linear, score=0.500, total=   0.0s
[CV]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=linear, score=0.800, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=linear ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=linear, score=0.444, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=linear ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=linear, score=0.556, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=linear ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=linear, score=0.222, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=sigmoid .........
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=sigmoid, score=0.200, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=sigmoid .........
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=sigmoid, score=0.200, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=sigmoid .........
[CV]  C=0.1, class_weight=balanced, ga

[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf, score=0.000, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf, score=0.200, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf, score=0.000, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf, score=0.222, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf ..........
[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=rbf, score=0.111, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=linear .......
[CV]  C=0.1, class_weight=balanced, gamma=0.0001, kernel=linear, score=0.500, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.0001, kernel=linear .......
[CV]  C=0.1, class_weight=balanced, g

[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=sigmoid, score=0.111, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=poly ..............
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=poly, score=0.600, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=poly ..............
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=poly, score=0.700, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=poly ..............
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=poly, score=0.778, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=poly ..............
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=poly, score=0.444, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=poly ..............
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=poly, score=0.222, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.01, kernel=rbf ..............
[CV]  C=1, class_weight=balanced, gamma=0.01, kernel=rbf, sco

...

[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=sigmoid, score=0.444, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=sigmoid ........
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=sigmoid, score=0.333, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=poly ...........
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=poly, score=0.600, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=poly ...........
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=poly, score=0.700, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=poly ...........
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=poly, score=0.667, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=poly ...........
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=poly, score=0.444, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=poly ...........
[CV]  C=100, class_weight=balanced, gamm

[CV]  C=1000, class_weight=balanced, gamma=1, kernel=linear, score=0.444, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=linear ...........
[CV]  C=1000, class_weight=balanced, gamma=1, kernel=linear, score=0.333, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=sigmoid ..........
[CV]  C=1000, class_weight=balanced, gamma=1, kernel=sigmoid, score=0.200, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=sigmoid ..........
[CV]  C=1000, class_weight=balanced, gamma=1, kernel=sigmoid, score=0.200, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=sigmoid ..........
[CV]  C=1000, class_weight=balanced, gamma=1, kernel=sigmoid, score=0.111, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=sigmoid ..........
[CV]  C=1000, class_weight=balanced, gamma=1, kernel=sigmoid, score=0.111, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=1, kernel=sigmoid ..........
[CV]  C=1000, class_weight=balanced, gamma

[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=rbf, score=0.444, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=rbf ..........
[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=rbf, score=0.333, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=linear .......
[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=linear, score=0.600, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=linear .......
[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=linear, score=0.700, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=linear .......
[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=linear, score=0.667, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=linear .......
[CV]  C=1000, class_weight=balanced, gamma=0.001, kernel=linear, score=0.444, total=   0.0s
[CV] C=1000, class_weight=balanced, gamma=0.001, kernel=linear .......
[CV]  C=1000, class_weight=b

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    3.7s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'sigmoid', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [225]:
# param_distributions = {"C": uniform(0, 1),'max_iter':uniform(1000,10000)}

# svm_clf = svm.LinearSVC()    
# rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=100, verbose=2, cv=3)
# rnd_search_cv.fit(hidden_train, train_semisup_label)                            

In [210]:
rnd_search_cv.best_score_


0.6402777777777778

In [232]:
grid.best_estimator_

SVC(C=100, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [233]:
best_clf = grid.best_estimator_

best_clf.fit(hidden_train, train_semisup_label)
test_pred_label = best_clf.predict(hidden_test)

score_report = classification_report(y_true=test_true_label, y_pred =test_pred_label)
print(score_report)
alarm_report = falseAnomRateCalc(test_true_label,test_pred_label)

2021-03-23 00:50:13,026 INFO    Total misclassified normal runs: 21, Total normal runs 913 
2021-03-23 00:50:13,027 INFO    0.023001095290251915
2021-03-23 00:50:13,027 INFO    Total misclassified anom runs: 125, Total anom runs 2227 
2021-03-23 00:50:13,028 INFO    0.05612932195779075


              precision    recall  f1-score   support

           0       0.88      0.98      0.92       913
           1       0.46      0.38      0.42       559
           2       0.37      0.43      0.39       559
           3       0.37      0.43      0.40       556
           4       1.00      0.65      0.79       553

    accuracy                           0.62      3140
   macro avg       0.61      0.57      0.58      3140
weighted avg       0.64      0.62      0.62      3140

